<a href="https://colab.research.google.com/github/HARDIK218/Large-Language-Model/blob/main/reway_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U torch datasets transformers tensorflow langchain playwright html2text sentence_transformers faiss-cpu
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.2/37.2 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.

In [ ]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    pipeline
)
from datasets import load_dataset
from peft import LoraConfig, PeftModel

from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain

In [ ]:
#################################################################
# Tokenizer
#################################################################

model_name='mistralai/Mistral-7B-Instruct-v0.1'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

In [ ]:
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
!playwright install
!playwright install-deps

153.1 Mb [] 0% 0.0s153.1 Mb [] 0% 29.5s153.1 Mb [] 0% 22.2s153.1 Mb [] 0% 13.1s153.1 Mb [] 0% 7.8s153.1 Mb [] 1% 6.8s153.1 Mb [] 1% 5.7s153.1 Mb [] 2% 5.1s153.1 Mb [] 2% 4.8s153.1 Mb [] 3% 4.6s153.1 Mb [] 4% 4.0s153.1 Mb [] 4% 4.1s153.1 Mb [] 5% 3.9s153.1 Mb [] 5% 4.0s153.1 Mb [] 5% 4.2s153.1 Mb [] 5% 4.4s153.1 Mb [] 5% 5.4s153.1 Mb [] 5% 5.5s153.1 Mb [] 6% 5.3s153.1 Mb [] 7% 5.2s153.1 Mb [] 7% 5.3s153.1 Mb [] 7% 5.1s153.1 Mb [] 8% 5.0s153.1 Mb [] 9% 4.9s153.1 Mb [] 10% 4.7s153.1 Mb [] 10% 4.6s153.1 Mb [] 11% 4.4s153.1 Mb [] 12% 4.4s153.1 Mb [] 12% 4.5s153.1 Mb [] 13% 4.5s153.1 Mb [] 13% 4.3s153.1 Mb [] 14% 4.3s153.1 Mb [] 15% 4.2s153.1 Mb [] 15% 4.1s153.1 Mb [] 16% 4.0s153.1 Mb [] 17% 3.9s153.1 Mb [] 17% 3.8s153.1 Mb [] 19% 3.6s153.1 Mb [] 20% 3.4s153.1 Mb [] 21% 3.3s153.1 Mb [] 22% 3.2s153.1 Mb [] 23% 3.0s153.1 Mb [] 24% 2.9s153.1 Mb [] 25% 2.8s153.1 Mb [] 26% 2.8s153.1 Mb [] 27% 2.7s153.1 Mb [] 28% 2.6s153.1 Mb [] 29% 2.6s153.1 Mb [] 30% 2.5s153.1 Mb [] 31% 2.5s153.1 Mb [] 32% 2.4s1

In [ ]:
# # NO NEED TO RUN AGAIN
import nest_asyncio
nest_asyncio.apply()

articles=[
    "https://en.wikipedia.org/wiki/Electronic_waste_recycling",
    "https://www.treehugger.com/what-is-e-waste-and-why-is-it-a-problem-5186270",
#     "https://indianexpress.com/article/world/climate-change/recycling-gone-up-last-5-years-67-e-waste-remains-unprocessed-8530613/",
#     "https://recykal.com/blog/growing-concern-about-e-waste-in-india/",
#     "https://www.downtoearth.org.in/blog/waste/recycling-of-e-waste-in-india-and-its-potential-64034",
#     'https://www.sciencedirect.com/topics/earth-and-planetary-sciences/extended-producer-responsibility#:~:text=EPR%20is%20a%20policy%20strategy,and%20environmental%20impact%20in%20general',
#     'https://pib.gov.in/PressReleasePage.aspx?PRID=1799170',
#     'https://recykal.com/blog/a-guide-to-epr-compliance-in-india/',
#     'https://www.shaktiplasticinds.com/extended-producer-responsibility-in-india-epr/',
#     'https://www.ewaste1.com/what-is-e-waste/',
#     'https://www.meity.gov.in/writereaddata/files/EWaste_Sep11_892011.pdf',
#     'https://www.drishtiias.com/daily-updates/daily-news-analysis/e-waste-management-in-india',
#     'https://hindrise.org/resources/e-waste-management-in-india/',
#     'https://recykal.com/blog/growing-concern-about-e-waste-in-india/',
#     'https://recykal.com/blog/how-to-safely-dispose-your-organizations-it-waste/',
#     'https://recykal.com/blog/differences-between-it-asset-disposal-vs-it-asset-disposition/',
#     'https://recykal.com/blog/why-you-need-to-meet-the-cpcb-deadline-for-filing-epr-plastic-returns/',
#     'https://recykal.com/blog/importance-of-epr-compliance-for-producers-importers-and-brand-owners/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-tyre-waste/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-e-waste/',
#     'https://recykal.com/blog/how-businesses-can-ensure-continuous-compliance-with-epr-regulations/',
#     'https://recykal.com/blog/who-should-have-an-epr-certificate/',
#     'https://recykal.com/blog/responsibilities-of-importers-for-epr-under-pwm/',
#     'https://recykal.com/blog/responsibilities-of-brands-for-epr-under-pwm/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-plastic-waste/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-battery-waste/',
#     'https://recykal.com/blog/why-every-business-must-have-an-it-asset-disposal-strategy/',
#     'https://recykal.com/blog/why-not-to-resell-your-organizations-used-it-assets/',
#     'https://recykal.com/blog/major-itam-risks-every-business-must-be-aware-of/',
#     'https://recykal.com/blog/what-are-the-benefits-of-it-asset-disposal/',
#     'https://recykal.com/blog/what-are-the-risks-of-it-asset-disposal/',
#     'https://recykal.com/blog/what-is-an-epr-certificate/',
#     'https://recykal.com/blog/what-is-the-cost-of-an-epr-certificate/',
#     'https://recykal.com/blog/how-do-i-get-an-epr-certificate/',
#     'https://recykal.com/blog/why-industrial-waste-management-is-important-for-business/',
#     'https://recykal.com/blog/why-plastic-neutrality-is-important-for-d2c-brands/',
#     'https://recykal.com/blog/all-you-need-to-know-about-plastic-neutrality/',
#     'https://recykal.com/blog/a-guide-to-epr-compliance-in-india/',
#     'https://recykal.com/blog/latest-epr-guidelines-for-tyre/',
#     'https://recykal.com/blog/why-are-all-d2c-brands-racing-towards-plastic-neutrality/',
#     'https://recykal.com/blog/exploring-the-challenges-and-opportunities-of-plastic-credits/',
#     'https://recykal.com/blog/an-introduction-to-plastic-credits/',
#     'https://recykal.com/blog/how-can-organizations-reduce-their-laptops-carbon-footprint/',
#     'https://recykal.com/blog/single-use-plastic-ban-what-will-change-from-july-1/',
#     'https://recykal.com/blog/what-is-industrial-waste/',
#     'https://recykal.com/blog/epr-and-sustainability-as-revenue-catalyst-2/',
#     'https://recykal.com/blog/pwm-rules-secondamendment-2021/',
#     'https://recykal.com/blog/pwm-rules-secondamendment-2021-2/',
#     'https://recykal.com/blog/plastic-waste-management-rules-amendment/',
#     'https://recykal.com/blog/plastic-recycling-epr-fulfillment/',
#     'https://recykal.com/blog/extended-producer-responsibility-status-around-the-world/',
#     'https://recykal.com/blog/how-is-plastic-waste-resource/',
#     'https://recykal.com/blog/epr-certification-mandatory-india/',
#     'https://recykal.com/blog/epr-registration-plastic-waste/',
#     'https://recykal.com/blog/pros-role-of-stakeholders-in-epr/',
#     'https://recykal.com/blog/recyclers-role-of-stakeholders-in-epr/',
#     'https://recykal.com/blog/an-introduction-to-e-waste/',
#     'https://recykal.com/blog/role-stakeholders-in-epr-producers-importers-brandowners/',
#     'https://recykal.com/blog/role-of-stakeholders-in-epr-consumers/',
#     'https://recykal.com/blog/stakeholders-in-epr-national-authority/',
#     'https://recykal.com/blog/global-plastic-pollution-epr/',
#     'https://recykal.com/blog/epr-challenges-in-india/',
#     'https://recykal.com/blog/waste-management-steps-bulk-waste-generators/',
#     'https://recykal.com/blog/plastic-waste-management-epr-fulfilment-2/',
#     'https://recykal.com/blog/epr-benefits-brands/',
#     'https://recykal.com/blog/cpcb-amends-sop-pibos-under-pwm-rules/',
#     'https://recykal.com/blog/data-quality-for-epr/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-action-plan-2/',
#     'https://recykal.com/blog/plastic-waste-management-epr-fulfilment/',
#     'http://swachhbharaturban.gov.in/writereaddata/SBM%20Plastic%20Waste%20Book.pdf',
#     'http://swachhbharaturban.gov.in/writereaddata/SBM%20Plastic%20Waste%20Book.pdf',
#     'https://www.shaktiplasticinds.com/how-to-manage-plastic-waste/',
#     'https://www.shaktiplasticinds.com/what-are-the-methods-of-plastic-waste-management/',
#     'https://timesofindia.indiatimes.com/blogs/voices/plastic-waste-and-its-management-strategies-for-environmental-sustainability/',
#     'https://www.drishtiias.com/daily-updates/daily-news-analysis/battery-waste-management-rules-2022',
#     'https://www.drishtiias.com/daily-updates/daily-news-analysis/battery-waste-management-and-handling-rules-2022',
#     'http://www.eprbatterycpcb.in/upload/adminDoc/Frequently%20Asked%20Questions%20(General).pdf',
#     'https://www.karosambhav.com/epr-compliance-for-battery-waste',
#     'https://www.corpseed.com/service/epr-for-waste-tyres',
#     'https://ssrana.in/articles/amendment-rules-extended-producer-responsibility-waste-tyre/',
#     'https://www.professionalutilities.com/epr-registration-for-tyre-waste/delhi.php',
#     'https://www.sciencedirect.com/science/article/abs/pii/S0141391021002809',
#     'https://link.springer.com/article/10.1007/s10163-022-01554-y',
#     'https://www.environmentalpollution.in/waste-management/rubber-industry/waste-from-rubber-industry-and-its-disposal/6938',
#     'https://climatepromise.undp.org/news-and-stories/what-is-circular-econ omy-and-how-it-helps-fight-climate-change',
#     'https://www.sciencedirect.com/topics/social-sciences/circular-economy',
#     'https://www.sustainability.com/thinking/creating-a-circular-economy-for-plastics/',
#     'https://www.sciencedirect.com/topics/earth-and-planetary-sciences/extended-producer-responsibility#:~:text=EPR%20is%20a%20policy%20strategy,and%20environmental%20impact%20in%20general',
#     'https://pib.gov.in/PressReleasePage.aspx?PRID=1799170',
#     'https://recykal.com/blog/a-guide-to-epr-compliance-in-india/',
#     'https://www.shaktiplasticinds.com/extended-producer-responsibility-in-india-epr/',
#     'https://www.ewaste1.com/what-is-e-waste/',
#     'https://www.meity.gov.in/writereaddata/files/EWaste_Sep11_892011.pdf',
#     'https://www.drishtiias.com/daily-updates/daily-news-analysis/e-waste-management-in-india',
#     'https://hindrise.org/resources/e-waste-management-in-india/',
#     'https://recykal.com/blog/growing-concern-about-e-waste-in-india/',
#     'https://recykal.com/blog/how-to-safely-dispose-your-organizations-it-waste/',
#     'https://recykal.com/blog/differences-between-it-asset-disposal-vs-it-asset-disposition/',
#     'https://recykal.com/blog/why-you-need-to-meet-the-cpcb-deadline-for-filing-epr-plastic-returns/',
#     'https://recykal.com/blog/importance-of-epr-compliance-for-producers-importers-and-brand-owners/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-tyre-waste/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-e-waste/',
#     'https://recykal.com/blog/how-businesses-can-ensure-continuous-compliance-with-epr-regulations/',
#     'https://recykal.com/blog/who-should-have-an-epr-certificate/',
#     'https://recykal.com/blog/responsibilities-of-importers-for-epr-under-pwm/',
#     'https://recykal.com/blog/responsibilities-of-brands-for-epr-under-pwm/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-plastic-waste/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-compliance-for-battery-waste/',
#     'https://recykal.com/blog/why-every-business-must-have-an-it-asset-disposal-strategy/',
#     'https://recykal.com/blog/why-not-to-resell-your-organizations-used-it-assets/',
#     'https://recykal.com/blog/major-itam-risks-every-business-must-be-aware-of/',
#     'https://recykal.com/blog/what-are-the-benefits-of-it-asset-disposal/',
#     'https://recykal.com/blog/what-are-the-risks-of-it-asset-disposal/',
#     'https://recykal.com/blog/what-is-an-epr-certificate/',
#     'https://recykal.com/blog/what-is-the-cost-of-an-epr-certificate/',
#     'https://recykal.com/blog/how-do-i-get-an-epr-certificate/',
#     'https://recykal.com/blog/why-industrial-waste-management-is-important-for-business/',
#     'https://recykal.com/blog/why-plastic-neutrality-is-important-for-d2c-brands/',
#     'https://recykal.com/blog/all-you-need-to-know-about-plastic-neutrality/',
#     'https://recykal.com/blog/a-guide-to-epr-compliance-in-india/',
#     'https://recykal.com/blog/latest-epr-guidelines-for-tyre/',
#     'https://recykal.com/blog/why-are-all-d2c-brands-racing-towards-plastic-neutrality/',
#     'https://recykal.com/blog/exploring-the-challenges-and-opportunities-of-plastic-credits/',
#     'https://recykal.com/blog/an-introduction-to-plastic-credits/',
#     'https://recykal.com/blog/how-can-organizations-reduce-their-laptops-carbon-footprint/',
#     'https://recykal.com/blog/single-use-plastic-ban-what-will-change-from-july-1/',
#     'https://recykal.com/blog/what-is-industrial-waste/',
#     'https://recykal.com/blog/epr-and-sustainability-as-revenue-catalyst-2/',
#     'https://recykal.com/blog/pwm-rules-secondamendment-2021/',
#     'https://recykal.com/blog/pwm-rules-secondamendment-2021-2/',
#     'https://recykal.com/blog/plastic-waste-management-rules-amendment/',
#     'https://recykal.com/blog/plastic-recycling-epr-fulfillment/',
#     'https://recykal.com/blog/extended-producer-responsibility-status-around-the-world/',
#     'https://recykal.com/blog/how-is-plastic-waste-resource/',
#     'https://recykal.com/blog/epr-certification-mandatory-india/',
#     'https://recykal.com/blog/epr-registration-plastic-waste/',
#     'https://recykal.com/blog/pros-role-of-stakeholders-in-epr/',
#     'https://recykal.com/blog/recyclers-role-of-stakeholders-in-epr/',
#     'https://recykal.com/blog/an-introduction-to-e-waste/',
#     'https://recykal.com/blog/role-stakeholders-in-epr-producers-importers-brandowners/',
#     'https://recykal.com/blog/role-of-stakeholders-in-epr-consumers/',
#     'https://recykal.com/blog/stakeholders-in-epr-national-authority/',
#     'https://recykal.com/blog/global-plastic-pollution-epr/',
#     'https://recykal.com/blog/epr-challenges-in-india/',
#     'https://recykal.com/blog/waste-management-steps-bulk-waste-generators/',
#     'https://recykal.com/blog/plastic-waste-management-epr-fulfilment-2/',
#     'https://recykal.com/blog/epr-benefits-brands/',
#     'https://recykal.com/blog/cpcb-amends-sop-pibos-under-pwm-rules/',
#     'https://recykal.com/blog/data-quality-for-epr/',
#     'https://recykal.com/blog/all-you-need-to-know-about-epr-action-plan-2/',
#     'https://recykal.com/blog/plastic-waste-management-epr-fulfilment/',
#     'http://swachhbharaturban.gov.in/writereaddata/SBM%20Plastic%20Waste%20Book.pdf',
#     'http://swachhbharaturban.gov.in/writereaddata/SBM%20Plastic%20Waste%20Book.pdf',
#     'https://www.shaktiplasticinds.com/how-to-manage-plastic-waste/',
#     'https://www.shaktiplasticinds.com/what-are-the-methods-of-plastic-waste-management/',
#     'https://timesofindia.indiatimes.com/blogs/voices/plastic-waste-and-its-management-strategies-for-environmental-sustainability/',
#     'https://www.drishtiias.com/daily-updates/daily-news-analysis/battery-waste-management-rules-2022',
#     'https://www.drishtiias.com/daily-updates/daily-news-analysis/battery-waste-management-and-handling-rules-2022',
#     'http://www.eprbatterycpcb.in/upload/adminDoc/Frequently%20Asked%20Questions%20(General).pdf',
#     'https://www.karosambhav.com/epr-compliance-for-battery-waste',
#     'https://www.corpseed.com/service/epr-for-waste-tyres',
#     'https://ssrana.in/articles/amendment-rules-extended-producer-responsibility-waste-tyre/',
#     'https://www.professionalutilities.com/epr-registration-for-tyre-waste/delhi.php',
#     'https://www.sciencedirect.com/science/article/abs/pii/S0141391021002809',
#     'https://link.springer.com/article/10.1007/s10163-022-01554-y',
#     'https://www.environmentalpollution.in/waste-management/rubber-industry/waste-from-rubber-industry-and-its-disposal/6938',
#     'https://climatepromise.undp.org/news-and-stories/what-is-circular-economy-and-how-it-helps-fight-climate-change',
#     'https://www.sciencedirect.com/topics/social-sciences/circular-economy',
#     'https://www.sustainability.com/thinking/creating-a-circular-economy-for-plastics/'
]

loader = AsyncChromiumLoader(articles)
docs = loader.load()

In [ ]:
type(docs)

list

In [ ]:
type(docs[0])

langchain_core.documents.base.Document

In [ ]:
# docs[0]

In [ ]:
# #NO NEED TO RUN AGAIN
# # Converts HTML to plain text
# html2text = Html2TextTransformer()
# docs_transformed = html2text.transform_documents(docs)

# # Chunk text
text_splitter = CharacterTextSplitter(chunk_size=500,
                                      chunk_overlap=0)
# chunked_documents = text_splitter.split_documents(docs_transformed)

# # Load chunked documents into the FAISS index
# # db = FAISS.from_documents(chunked_documents,
# #                           HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

# # retriever = db.as_retriever()

In [ ]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 9.6 MB/s eta 0:00:00


In [ ]:
from langchain.vectorstores import Pinecone
import pinecone
pinecone.init(
    api_key='e081611d-59ab-42d4-adc7-a0ce304b22db',
    environment='gcp-starter'
)
index_name = 'reway-mistral-llm'

In [ ]:
index = pinecone.Index(index_name)
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.08132,
 'namespaces': {'': {'vector_count': 8132}},
 'total_vector_count': 8132}

In [ ]:
embed = HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-mpnet-base-v2'
    )
text_field="text"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)
retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/langchain_community/vectorstores/pinecone.py:62: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [ ]:
import pandas as pd
import csv
df = pd.read_csv('QA_dataset.csv')
df.head()
qa = []

filename = 'QA_dataset.csv'

with open(filename, 'r') as csvfile:
    datareader = csv.reader(csvfile)
    for row in datareader:
        str=''
        for col in row:
          str+=col
          str+=' '
        qa.append(str)

qa.pop(0)

'Question Answer '

In [ ]:
qa[:5]

["What is Electrical and Electronic Equipment (EEE)? Under the E-Waste (M) Rules, 2016, 'Electrical and Electronic Equipment' (EEE) means\nequipment which are dependent on electric current or electro-magnetic field in order to\nbecome functional. ",
 'What Is E- Waste?  Electronic Waste (E-Waste) means electrical and electronic equipment, whole or in part\ndiscarded as waste by the consumer or bulk consumer as well as rejects from manufacturing,\nrefurbishment and repair processes.\nIt comprises of end of life information technology and telecommunication (IT & Telecoms)\nequipment such as centralized data processing, mainframes, minicomputers, personal\ncomputers, laptops, printers, use terminals, cellular phone, etc. and end of life consumer\nelectrical and electronics such as television sets, refrigerator, air conditioner, washing\nmachine and fluorescent and other mercury containing lamps ',
 'What is general composition of E-Waste? . E-waste contains useful material of economic ben

In [ ]:
from langchain.schema import Document

In [ ]:
qa_final = []
it=1
for item in qa:
  qa_final.append(Document(page_content=item, metadata={'id':it},))
  it+=1

qa_final[:5]

[Document(page_content="What is Electrical and Electronic Equipment (EEE)? Under the E-Waste (M) Rules, 2016, 'Electrical and Electronic Equipment' (EEE) means\nequipment which are dependent on electric current or electro-magnetic field in order to\nbecome functional. ", metadata={'id': 1}),
 Document(page_content='What Is E- Waste?  Electronic Waste (E-Waste) means electrical and electronic equipment, whole or in part\ndiscarded as waste by the consumer or bulk consumer as well as rejects from manufacturing,\nrefurbishment and repair processes.\nIt comprises of end of life information technology and telecommunication (IT & Telecoms)\nequipment such as centralized data processing, mainframes, minicomputers, personal\ncomputers, laptops, printers, use terminals, cellular phone, etc. and end of life consumer\nelectrical and electronics such as television sets, refrigerator, air conditioner, washing\nmachine and fluorescent and other mercury containing lamps ', metadata={'id': 2}),
 Docum

In [ ]:
texts = [d.page_content for d in qa_final]

In [ ]:
# DO NOT RUN AGAIN
docsearch = Pinecone.from_documents(qa_final, HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'), index_name=index_name)
retriever = docsearch.as_retriever()

In [ ]:
# query = 'What is the importance of e-waste?'
# docs = docsearch.similarity_search(query, include_metadeta=False)

In [ ]:
prompt_template = """
### [INST] Instruction: Answer the question based on your e-waste knowledge. Here is context to help:

{context}

### QUESTION:
{question} [/INST]
"""

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [ ]:
rag_chain = (
 {"context": retriever, "question": RunnablePassthrough()}
    | llm_chain
)

result = rag_chain.invoke("What is the impact of e-waste on environment?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result['context']

[Document(page_content='Why is E-waste management important?\n E-waste management is important to keep electrical and electronic waste out of landfills. Damaged computers, laptops, mobile devices, televisions, and even household appliances are considered to be E-waste. When improperly disposed of, the toxic elements in e-waste can pollute the environment. That is why e waste recycling can significantly reduce the increasing risks of greenhouse gas emissions. ', metadata={'id': 81.0}),
 Document(page_content='Mobile phones, Global Positioning Systems (GPS), routers,  \npocket calculators, personal computers  \nprinters, telephones| Office and Medical Equipment  \n---|---  \nCopiers/Printers| Dialysis Machines  \nWiFi Dongles| Imaging Equipment  \nIT Servers and Server Racks| Defibrillator  \nPhone & PBX systems| Power Distribution Systems (PDUs)  \nAudio & Video Equipment| Autoclave  \nNetwork Hardware (i.e. servers, switches, hubs, etc.)| Cords and Cables  \nPower Strips & Power Suppli

In [ ]:
print(result['text'])


The impact of e-waste on the environment can be significant if it is not properly managed. Improper disposal of e-waste can lead to pollution of the environment with toxic elements such as mercury, lead, cadmium, and brominated flame retardants. These elements can contaminate soil, water, and air, leading to negative impacts on human health and the environment. Additionally, e-waste contains valuable materials such as gold, silver, and copper, which can be recovered through recycling and reused in new products. However, if e-waste is not properly recycled, these valuable resources are lost forever. Furthermore, the production of new electronic products requires the extraction of raw materials, which can have negative environmental impacts. Therefore, proper e-waste management is crucial for reducing the negative impacts on the environment and protecting human health.
